In [2]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import spacy

### Reading our editing Amazon Dataset.

In [3]:
amazon_ds = pd.read_csv("A_amazon_catelogue.csv", encoding = 'Latin-1')
amazon_ds.head()

,ID,ASIN,item_name
0,NaN,B09TLGFJCV,A & D - Strip of 10 Capsules
1,NaN,B0BB6DG4QD,A Bec L - Bottle of 30 Tablets
2,NaN,B0BB6KKQDS,A BEC TABLET
3,NaN,B08BXDM8KL,A Doxid - Strip of 10 Tablets
4,NaN,B0947JD4BX,A Doxid L - Strip of 10 Capsules


### Reading our product master Dataset.

In [4]:
pd_master = pd.read_csv("A_product_master_2.csv")
pd_master.head()

,Product_ID,Client_ID,Product_Code,Product_Desc,UOM,Packsize,Manufacturer,MRP,PTS,PTR,...,Category,molecular_id,P_ID,GST_PER,Manufacturer_Code,ED_MDM_Product_Code,ED_MDM_Product_Image,ED_MDM_Product_Document_Image,ED_MDM_Product_Creation_Time,Match
0,1697255,756,J0002111,A MAX CAPSULES 10^S,NaN,10'S,DERMAWIZ LAB DERMA,155.0,108.50,108.50,...,NaN,NaN,117107987.0,NaN,1h,ED000157592,NaN,NaN,NaN,NaN
1,1696011,756,J0002112,A PLUS TABLETS,NaN,NaN,MEDLIFE GENETICS-LAM,230.0,0.00,155.90,...,NaN,NaN,NaN,NaN,GXN,ED000148859,NaN,NaN,NaN,NaN
2,1714647,756,J0002113,A TO Z GOLD CAPSULES NS 15CAPSULES,NaN,15CAP,ALKEM #M CARE#,210.0,142.37,142.39,...,NaN,NaN,117111597.0,NaN,BB,ED000000174,NaN,NaN,NaN,NaN
3,1686756,756,J0002115,A TO Z GOLD CAPSULES 15^S,NaN,15'S,ALKEM ALTIS,210.0,128.14,142.37,...,NaN,NaN,117124974.0,NaN,GXQ,ED000155268,NaN,NaN,NaN,NaN
4,7790857,756,A00849,A TO Z IMMUNE TABLETS 1*16,NaN,1*16,ALKEM #M CARE#,215.0,145.76,145.76,...,NaN,NaN,118559972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
amazon_ds['item_name']

0                            A & D - Strip of 10 Capsules
1                          A Bec L - Bottle of 30 Tablets
2                                            A BEC TABLET
3                           A Doxid - Strip of 10 Tablets
4                        A Doxid L - Strip of 10 Capsules
                              ...                        
4280                    Azuvas 40MG - Strip of 10 Tablets
4281                   Azysafe 500Mg - Strip of 5 Tablets
4282                     Azysafe-500 - Strip of 5 Tablets
4283                   AZYTIME 500MG - Strip of 3 Tablets
4284    Azzura Essential Renal Nutrition Vanilla Flavo...
Name: item_name, Length: 4285, dtype: object

## In amazon Dataframe.

### Spliting the "item_name" column along "-".

In [6]:
for items1 in amazon_ds['item_name']:
    string = str(items1)
    last_dash_index = string.rfind("-")
    
    if last_dash_index != -1:
        first_part = string[:last_dash_index]
        second_part = string[last_dash_index + 1:]
        dict1 = {first_part: second_part}
    else:
        string1 = string.replace(" ", "")
#         print(string1)
    keys = list(dict1.keys())  # Convert dict_keys object to a list
    
    for i in keys:
        i = i.replace(" ", "")
#         print(i)
    

### Assigning two columns for the medicine name and the type.

In [7]:
df = amazon_ds[['item_name']] 
df[['First_Part', 'Type']] = df['item_name'].str.rsplit('-', n=1, expand=True)

df['First_Part'] = df['First_Part'].str.replace(" ", "")
df['First_Part'] = df['First_Part'].str.upper()

df.head()

C:\Users\mohammad\AppData\Local\Temp\ipykernel_16376\1495230820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['First_Part', 'Type']] = df['item_name'].str.rsplit('-', n=1, expand=True)


,item_name,First_Part,Type
0,A & D - Strip of 10 Capsules,A&D,Strip of 10 Capsules
1,A Bec L - Bottle of 30 Tablets,ABECL,Bottle of 30 Tablets
2,A BEC TABLET,ABECTABLET,None
3,A Doxid - Strip of 10 Tablets,ADOXID,Strip of 10 Tablets
4,A Doxid L - Strip of 10 Capsules,ADOXIDL,Strip of 10 Capsules


In [8]:
df.shape

(4285, 3)

## In Product Master Dataframe.

### 

In [9]:
df_2 = pd.read_csv("File.csv")

In [10]:
df_2.head()

,Unnamed: 0,Product_Desc,First_part,Type
0,0,A MAX CAPSULES 10^S,AMAX,CAPSULES Strip of 10
1,1,A PLUS TABLETS,APLUS,TABLETS
2,2,A TO Z GOLD CAPSULES NS 15CAPSULES,ATOZGOLD,CAPSULES
3,3,A TO Z GOLD CAPSULES 15^S,ATOZGOLD,CAPSULES Strip of 15
4,4,A TO Z IMMUNE TABLETS 1*16,ATOZIMMUNE,TABLETS 1*16


In [11]:
df_2.shape

(3449, 4)

In [12]:
for x in df["First_Part"]:
    for y in df_2["First_part"]:
        similarity_score = fuzz.ratio(x,y)
        if similarity_score >= 85:
            print(x, "------", y, similarity_score)

ASTAR-P ------ ASTARP 92
ASTAR-P ------ ASTARP 92
ATOZGOLD ------ ATOZGOLD 100
ATOZGOLD ------ ATOZGOLD 100
ATOZIMMUNE ------ ATOZIMMUNE 100
ATOZIMMUNE ------ ATOZIMMUNE 100
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS 100
ATOZWOMEN ------ ATOZWOMAN 89
ATRET10MG ------ ACUTRET10MG 90
ATRET10MG ------ ACUTRET10MG 90
ATRET10MG ------ ACUTRET10MG 90
AACTARIL ------ AACTARIL 

ACIVIR ------ ACIVIR 100
ACIVIR ------ ACIVIR 100
ACIVIR ------ ACIVIR 100
ACIVIR ------ ACIVIR 100
ACIVIRIV ------ ACIVIR 86
ACIVIRIV ------ ACIVIR 86
ACIVIR-200DT ------ ACIVIR200DT 96
ACIVIR-200DT ------ ACIVIR200DT 96
ACIVIR-200DT ------ ACIVIR400DT 87
ACIVIR-200DT ------ ACIVIR400DT 87
ACIVIR-200DT ------ ACIVIR400DT 87
ACIVIR-200DT ------ ACIVIR800DT 87
ACIVIR-200DT ------ ACIVIR800DT 87
ACIVIR-200DT ------ ACIVIR800DT 87
ACIVIR-400DT ------ ACIVIR200DT 87
ACIVIR-400DT ------ ACIVIR200DT 87
ACIVIR-400DT ------ ACIVIR400DT 96
ACIVIR-400DT ------ ACIVIR400DT 96
ACIVIR-400DT ------ ACIVIR400DT 96
ACIVIR-400DT ------ ACIVIR800DT 87
ACIVIR-400DT ------ ACIVIR800DT 87
ACIVIR-400DT ------ ACIVIR800DT 87
ACIVIR-800DT ------ ACIVIR200DT 87
ACIVIR-800DT ------ ACIVIR200DT 87
ACIVIR-800DT ------ ACIVIR400DT 87
ACIVIR-800DT ------ ACIVIR400DT 87
ACIVIR-800DT ------ ACIVIR400DT 87
ACIVIR-800DT ------ ACIVIR800DT 96
ACIVIR-800DT ------ ACIVIR800DT 96
ACIVIR-800DT ------ ACIVIR800DT 96
ACLAIR -

AERODILLS ------ AERODILLS 100
AEROLIFESPACER ------ AEROLIFESPACER1UNIT 85
AEROMONT-B ------ AEROMONTB 95
AEROMONT-B ------ AEROMONTB 95
AEROTROP ------ AEROTROP 100
AEROTROP ------ AEROTROPF 94
AEROTROPF ------ AEROTROP 94
AEROTROPF ------ AEROTROPF 100
AEROTROP-F ------ AEROTROP 89
AEROTROP-F ------ AEROTROPF 95
AF-C ------ AF-C 100
AFDERM-MN ------ AFDERMMN 94
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN 94
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDURA ------ AFDURA 100
AFENACP ------ ACENACP 86
AFENACP ------ ACENACP 86
AFENACP ------ AFENACP 100
AFENACP ------ ALFENAC 86
AFITRA-200 ------ AFITRA200 95
AF-KIT ------ AF-KIT4 92
AFLAB2 ------ AFLAB2 100
AFOGATRAN110 ------ AFOGATRAN110 100
AFOGATRAN110 ------ AFOGATRAN150 92
AFOGATRAN150 ------ AFOGATRAN110 92
AFOGATRAN150 ------ AFOGATRAN150 100
AFOGATRAN150 ------ AFOGATRAN75 87
AFOGATRAN75 ------ AFOGATRAN150 

ALEXP ------ ALEX 89
ALEXP ------ ALEXP 100
ALEXP ------ ALEX 89
ALEXP ------ ALEX 89
ALEXPLUS ------ ALEXLS 86
ALEXPLUS ------ ALEXLS 86
ALEXPLUS ------ ALEXLS 86
ALEXPLUS ------ ALEXLS 86
ALEXSUGARFREE ------ ALEXSUGARFREE 100
ALEX-LSF ------ ALEXLS 86
ALEX-P ------ ALEXP 91
ALFAGPC ------ ALFAGPC 100
ALFAOSTEBON ------ ALFAOSTEBON 100
ALFACALPLUS ------ ALFACALPLUS 100
ALFAGABA100MG ------ ALFAGABA100 92
ALFA-MAXICAL ------ ALFA-MAXICAL 100
ALFATAM-D ------ ALFATAMD 94
ALFENCE ------ ALFENAC 86
ALFOO10MG ------ ALFOO10 88
ALFOO10MG ------ ALFOO10 88
ALFUGRESS10MG ------ ALFUGRESS1*10 85
ALFUSIND ------ ALFUSIND# 94
ALFUSIND ------ ALFUSIND 100
ALFUSIND ------ ALFUSIN 93
ALFUSIND ------ ALFUSIN 93
ALGEST200 ------ ALGEST200 100
ALGEST200 ------ ALGESTSR200 90
ALGESTSR200 ------ ALGEST200 90
ALGESTSR200 ------ ALGESTSR200 100
ALGESTSR200 ------ ALGESTSR300 91
ALGESTSR300MG ------ ALGESTSR300 92
ALGOSSPAS ------ ALGOSSPAS 100
ALIVHER ------ ALIVHERES 88
ALIVHER ------ ALIVHER 100
ALIVH

AMARYL1MG ------ AMARYL1 88
AMARYL2MG ------ AMARYL2 88
AMARYL3MG ------ AMARYL3 88
AMARYLM1MG ------ AMARYLM1 89
AMARYLM1MG ------ AMARYLM1 89
AMARYLM1MG ------ AMARYLM1 89
AMARYLM1MG ------ AMARYLM1 89
AMARYLM2MG ------ AMARYLM2 89
AMARYLM2MG ------ AMARYLM2 89
AMARYLMFORTE1 ------ AMARYLM1FORTE 92
AMARYLMFORTE1 ------ AMARYLM2FORTE 92
AMARYLMFORTE1MG ------ AMARYLM1FORTE 86
AMARYLMFORTE1MG ------ AMARYLM2FORTE 86
AMARYLMFORTE2 ------ AMARYLM1FORTE 92
AMARYLMFORTE2 ------ AMARYLM2FORTE 92
AMARYLMFORTE2MG ------ AMARYLM1FORTE 86
AMARYLMFORTE2MG ------ AMARYLM2FORTE 86
AMARYLM2 ------ AMARYL2 93
AMARYLM2 ------ AMARYLM1 88
AMARYLM2 ------ AMARYLM1 88
AMARYLM2 ------ AMARYLM2 100
AMARYLM2 ------ AMARYLM2 100
AMARYLM2 ------ AMARYLMP2 94
AMARYLM2 ------ AMARYLMV2 94
AMARYLMP2 ------ AMARYL2 88
AMARYLMP2 ------ AMARYLM2 94
AMARYLMP2 ------ AMARYLM2 94
AMARYLMP2 ------ AMARYLMP2 100
AMARYLMP2 ------ AMARYLMV2 89
AMARYLMP2MG ------ AMARYLMP2 90
AMARYLMV1MG ------ AMARYLMV1 90
AMARYLMV2MG --

KeyboardInterrupt: 

In [13]:
df.head(20)

,item_name,First_Part,Type
0,A & D - Strip of 10 Capsules,A&D,Strip of 10 Capsules
1,A Bec L - Bottle of 30 Tablets,ABECL,Bottle of 30 Tablets
2,A BEC TABLET,ABECTABLET,None
3,A Doxid - Strip of 10 Tablets,ADOXID,Strip of 10 Tablets
4,A Doxid L - Strip of 10 Capsules,ADOXIDL,Strip of 10 Capsules
5,A K T FD - Strip of 3 Tablets,AKTFD,Strip of 3 Tablets
6,A K T FD - Strip of 3 Tablets,AKTFD,Strip of 3 Tablets
7,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
8,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
9,A To Z - Bottle of 100ml Syrup,ATOZ,Bottle of 100ml Syrup


In [18]:
index_1 = []
# index_2 = []
for index_1, x in (df["First_Part"]):
    for y in df_2["First_part"]:
        similarity_score = fuzz.ratio(x, y)
        if similarity_score >= 90:
            index_1.append(index_1)


ValueError: too many values to unpack (expected 2)

In [ ]:
match_indices_1 = []
match_indices_2 = []

for i, x in enumerate(df["First_Part"]):
    for j, y in df_2["First_part"].iteritems():
        similarity_score = fuzz.ratio(x, y)
        if similarity_score >= 90:
            match_indices_1.append(i)
            match_indices_2.append(j)

In [ ]:
new_df = pd.DataFrame(columns=df.columns)  # Create a new empty DataFrame with the same columns

for a in match_indices_1:
    row = df.iloc[a]  # Retrieve the row using iloc
    new_df = new_df.append(row)  # Append the row to the new DataFrame

# Now you can use the new_df DataFrame for further operations

In [31]:
index_1 = []
index_2 = []
for i, x in enumerate(df["First_Part"]):
    for j, y in enumerate(df_2["First_part"]):
        similarity_score = fuzz.ratio(x, y)
        if similarity_score >= 90:
            index_1.append(i)
            index_2.append(j)

In [52]:
new_df_1 = df.loc[index_1]
new_df.head(15)

,item_name,First_Part,Type
7,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
8,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
11,A To Z Gold - Strip of 15 Capsules,ATOZGOLD,Strip of 15 Capsules
11,A To Z Gold - Strip of 15 Capsules,ATOZGOLD,Strip of 15 Capsules
12,A To Z Immune - Strip of 16 Tablets,ATOZIMMUNE,Strip of 16 Tablets
12,A To Z Immune - Strip of 16 Tablets,ATOZIMMUNE,Strip of 16 Tablets
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup


In [40]:
new_df_2 = df_2.loc[index_2]
new_df_2.head(15)

,Unnamed: 0,Product_Desc,First_part,Type
2601,2601,ASTAR P TABLETS 1*10,ASTARP,TABLETS 1*10
2601,2601,ASTAR P TABLETS 1*10,ASTARP,TABLETS 1*10
2,2,A TO Z GOLD CAPSULES NS 15CAPSULES,ATOZGOLD,CAPSULES
3,3,A TO Z GOLD CAPSULES 15^S,ATOZGOLD,CAPSULES Strip of 15
4,4,A TO Z IMMUNE TABLETS 1*16,ATOZIMMUNE,TABLETS 1*16
5,5,A TO Z IMMUNE TABLETS 16^S,ATOZIMMUNE,TABLETS Strip of 16
6,6,A TO Z NS + DROP # 15ML,ATOZNS+,DROP # 15ML
7,7,A TO Z NS + DROP 5% 15ML,ATOZNS+,DROP 5% 15ML
8,8,A TO Z NS + DROPS 15ML,ATOZNS+,DROPS 15ML
9,9,A TO Z NS + TABLETS 1*15,ATOZNS+,TABLETS 1*15


In [55]:
for g in new_df_1["Type"]:
    g = str(g).upper().replace("TABLET", "TABLETS")
new_df_1["Type"] = g

In [56]:
new_df_1["Type"]

7        STRIP OF 10 TABLETS
8        STRIP OF 10 TABLETS
11       STRIP OF 10 TABLETS
11       STRIP OF 10 TABLETS
12       STRIP OF 10 TABLETS
                ...         
4263     STRIP OF 10 TABLETS
4263     STRIP OF 10 TABLETS
4264     STRIP OF 10 TABLETS
4272     STRIP OF 10 TABLETS
4276     STRIP OF 10 TABLETS
Name: Type, Length: 2213, dtype: object

In [62]:
for k in new_df_1["Type"]:
    for l in new_df_2["Type"]:
        similarity_Score_2 = fuzz.ratio(k,l)
        if similarity_Score_2 >= 1:
            print(k, "------------", l, similarity_Score_2)

 STRIP OF 10 TABLETS ------------ TABLETS 1*10 44
 STRIP OF 10 TABLETS ------------ TABLETS 1*10 44
 STRIP OF 10 TABLETS ------------ CAPSULES 29
 STRIP OF 10 TABLETS ------------ CAPSULES Strip of 15 25
 STRIP OF 10 TABLETS ------------ TABLETS 1*16 44
 STRIP OF 10 TABLETS ------------ TABLETS Strip of 16 36
 STRIP OF 10 TABLETS ------------ DROP # 15ML 39
 STRIP OF 10 TABLETS ------------ DROP 5% 15ML 38
 STRIP OF 10 TABLETS ------------ DROPS 15ML 33
 STRIP OF 10 TABLETS ------------ TABLETS 1*15 44
 STRIP OF 10 TABLETS ------------ TABLETS 15S 45
 STRIP OF 10 TABLETS ------------ TABLETS 15T 45
 STRIP OF 10 TABLETS ------------ TABLETS NEW 15.S 39
 STRIP OF 10 TABLETS ------------ SYRUP # 200ML 42
 STRIP OF 10 TABLETS ------------ SYRUP(MANGO) 200ML 37
 STRIP OF 10 TABLETS ------------ DROP # 15ML 39
 STRIP OF 10 TABLETS ------------ DROP 5% 15ML 38
 STRIP OF 10 TABLETS ------------ DROPS 15ML 33
 STRIP OF 10 TABLETS ------------ TABLETS 1*15 44
 STRIP OF 10 TABLETS ------------ TA

TypeError: object of type 'float' has no len()